In [3]:
%%pyspark
import pyspark 
print(print(pyspark.__version__)) 

2.4.4.dev0
None

In [5]:
spark.conf.set(
  "fs.azure.account.key.waginput.blob.core.windows.net",
  "TBwLoPOLim87APX5grtZzWy8Td9h69F/BJgDxuiQyEP480Cs5zyOa2bUeVVRfUnCALOug3aA2Wb4cj8aIiqGEw==")

In [6]:
val df = spark.read.option("header","true").option("inferSchema","true").csv("wasbs://incoming@waginput.blob.core.windows.net/train.csv")

df: org.apache.spark.sql.DataFrame = [key: timestamp, fare_amount: double ... 6 more fields]

In [7]:
display(df)

In [12]:
df.printSchema

root
 |-- key: timestamp (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)

In [15]:
df.head(10)

res8: Array[org.apache.spark.sql.Row] = Array([2009-06-15 17:26:21.0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.84161,40.712278,1], [2010-01-05 16:52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1], [2011-08-18 00:35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.76127,-73.991242,40.750562,2], [2012-04-21 04:30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.98713,40.733143,-73.991567,40.758092,1], [2010-03-09 07:51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1], [2011-01-06 09:50:45.0,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.73163,-73.972892,40.758233,1], [2012-11-20 20:35:00.0,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1], [2012-01-04 17:22:00.0,16.5,2012-01-04 17:22:00 UTC,-73.9513,40.774138,-73...

In [16]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
val featureCols=Array("fare_amount","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count")
val assembler: org.apache.spark.ml.feature.VectorAssembler= new VectorAssembler().setInputCols(featureCols).setOutputCol("features")

val assembledDF = assembler.setHandleInvalid("skip").transform(df)
val assembledFinalDF = assembledDF.select("fare_amount","features")

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
featureCols: Array[String] = Array(fare_amount, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, passenger_count)
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_007bc7e9c76f
assembledDF: org.apache.spark.sql.DataFrame = [key: timestamp, fare_amount: double ... 7 more fields]
assembledFinalDF: org.apache.spark.sql.DataFrame = [fare_amount: double, features: vector]

In [18]:
import org.apache.spark.ml.feature.Normalizer

val normalizedDF = new Normalizer().setInputCol("features").setOutputCol("normalizedFeatures").transform(assembledFinalDF)

import org.apache.spark.ml.feature.Normalizer
normalizedDF: org.apache.spark.sql.DataFrame = [fare_amount: double, features: vector ... 1 more field]

In [20]:
val normalizedDF1 = normalizedDF.na.drop()

normalizedDF1: org.apache.spark.sql.DataFrame = [fare_amount: double, features: vector ... 1 more field]

In [21]:
val Array(trainingDS, testDS) = normalizedDF1.randomSplit(Array(0.7, 0.3))

trainingDS: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [fare_amount: double, features: vector ... 1 more field]
testDS: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [fare_amount: double, features: vector ... 1 more field]

In [ ]:
trainingDS.count

In [22]:
import org.apache.spark.ml.regression.LinearRegression
// Create a LinearRegression instance. This instance is an Estimator.
val lr = new LinearRegression().setLabelCol("fare_amount").setMaxIter(100)
// Print out the parameters, documentation, and any default values.
println(s"Linear Regression parameters:\n ${lr.explainParams()}\n")
// Learn a Linear Regression model. This uses the parameters stored in lr.
val lrModel = lr.fit(trainingDS)
// Make predictions on test data using the Transformer.transform() method.
// LinearRegression.transform will only use the 'features' column.
val lrPredictions = lrModel.transform(testDS)

import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_0930c601db7b
Linear Regression parameters:
 aggregationDepth: suggested depth for treeAggregate (>= 2) (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. (default: 1.35)
featuresCol: features column name (default: features)
fitIntercept: whether to fit an intercept term (default: true)
labelCol: label column name (default: label, current: fare_amount)
loss: The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError) (default: squaredError)
maxIter: maximum number of iterations (>= 0) (default: 100, current: 100)
predictionCol: prediction column name (default: prediction)
regParam: regularization parameter (>= 0) (default: 0.0)
solver

In [23]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
println("\nPredictions : " )
lrPredictions.select($"fare_amount".cast(IntegerType),$"prediction".cast(IntegerType)).orderBy(abs($"prediction"-$"fare_amount")).distinct.show(15)

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

Predictions : 
+-----------+----------+
|fare_amount|prediction|
+-----------+----------+
|         49|        49|
|        331|       331|
|        259|       259|
|         15|        14|
|         -5|        -5|
|         51|        50|
|        315|       314|
|         41|        40|
|         91|        91|
|        132|       132|
|        230|       230|
|        286|       286|
|        169|       169|
|        205|       205|
|         26|        25|
+-----------+----------+
only showing top 15 rows

In [24]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
val evaluator_r2 = new RegressionEvaluator().setPredictionCol("prediction").setLabelCol("fare_amount").setMetricName("r2")

//As the name implies, isLargerBetter returns if a larger value is better or smaller for evaluation.
val isLargerBetter : Boolean = evaluator_r2.isLargerBetter
println("Coefficient of determination = " + evaluator_r2.evaluate(lrPredictions))

import org.apache.spark.ml.evaluation.RegressionEvaluator
evaluator_r2: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_793bcc025eab
isLargerBetter: Boolean = true
Coefficient of determination = 1.0

In [25]:
//Evaluate the results. Calculate Root Mean Square Error
val evaluator_rmse = new RegressionEvaluator().setPredictionCol("prediction").setLabelCol("fare_amount").setMetricName("rmse")
//As the name implies, isLargerBetter returns if a larger value is better for evaluation.
val isLargerBetter1 : Boolean = evaluator_rmse.isLargerBetter
println("Root Mean Square Error = " + evaluator_rmse.evaluate(lrPredictions))

evaluator_rmse: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_b89b4f6a81fb
isLargerBetter1: Boolean = false
Root Mean Square Error = 7.154196877704513E-15